Используемые библиотеки


In [1]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, List
import operator

from langchain_gigachat.chat_models import GigaChat
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END, START

load_dotenv()

True

Простейшая реализация подключения к Gigachat API

In [2]:
print("GIGACHAT_CLIENT_ID:", os.getenv("GIGACHAT_CLIENT_ID"))
print("GIGACHAT_CREDENTIALS:", os.getenv("GIGACHAT_CREDENTIALS"))
print("GIGACHAT_SCOPE:", os.getenv("GIGACHAT_SCOPE"))

def create_gigachat_parser_client():
    return GigaChat(
        credentials=os.getenv("GIGACHAT_CREDENTIALS"),
        verify_ssl_certs=False,
        scope=os.getenv("GIGACHAT_SCOPE", "GIGACHAT_API_PERS"),
        temperature=0.2,
        timeout=60
    )
def create_gigachat_quiz_client():
    return GigaChat(
        credentials=os.getenv("GIGACHAT_CREDENTIALS"),
        verify_ssl_certs=False,
        scope=os.getenv("GIGACHAT_SCOPE", "GIGACHAT_API_PERS"),
        temperature=0.6,
        timeout=60
    )

llm_parser = create_gigachat_parser_client()
llm_quiz = create_gigachat_quiz_client()
test_response_parser = llm_parser.invoke([HumanMessage(content="Привет! Ты работаешь?")])
test_response_quiz = llm_quiz.invoke([HumanMessage(content="Привет! Ты работаешь?")])
print("🤖 GigaChat ответил:", test_response_parser.content)
print("*"*100)
print("🤖 GigaChat ответил:", test_response_quiz.content)


GIGACHAT_CLIENT_ID: 019a1f79-b5b9-7735-9e39-f5eeb0918d30
GIGACHAT_CREDENTIALS: MDE5YTFmNzktYjViOS03NzM1LTllMzktZjVlZWIwOTE4ZDMwOjc4NGU3NzdjLTg5ZjQtNGZkMC1iYWIyLTkxOTg4NTQ3ZDIzMQ==
GIGACHAT_SCOPE: GIGACHAT_API_PERS
🤖 GigaChat ответил: Привет! Да, работаю — отвечаю на твои вопросы, помогаю решать задачи, обсуждать темы, писать тексты и многое другое. Что нужно обсудить?
****************************************************************************************************
🤖 GigaChat ответил: Привет! Да, работаю — отвечаю на твои вопросы, помогаю решать задачи, обсуждать темы и даже общаться на разные интересные темы. Что хочешь обсудить?


Парсер-агент

In [ ]:
    with open("sample_lecture.txt", "r", encoding="utf-8") as f:
        lecture = f.read()
    
    
    test_state = {
        "lecture_text": lecture,
        "key_facts": [],
        "quiz_questions": [],
        "messages": [],
        "current_step": "start"
    }
    def parser_agent(state: test_state) -> test_state:
        print("Парсер агент запущен...")
        lecture_text = state["lecture_text"]
        system_prompt = ChatPromptTemplate.from_messages([
        ("system", """Ты — эксперт-аналитик образовательного контента с многолетним опытом работы в академической среде.
    
        Твоя основная задача: тщательно проанализировать предоставленный текст лекции и извлечь наиболее значимые ключевые факты, которые студент должен запомнить для глубокого понимания темы.
    
        КРИТЕРИИ ОТБОРА ФАКТОВ:
        1. Полнота и самодостаточность — каждый факт представляет собой законченное, самостоятельное утверждение, которое можно понять вне контекста лекции
        2. Проверяемость и конкретность — факты должны содержать конкретные определения, формулы, даты, названия, числовые данные или четкие логические связи
        3. Приоритет важности — отбирай только концептуально значимую информацию: определения ключевых терминов, основные теоремы, законы, формулы, исторические вехи, причинно-следственные связи
        4. Точность формулировок — сохраняй терминологию и научную строгость из оригинального текста
        5. Избегай повторов — каждый факт должен нести уникальную информацию
    
        КОЛИЧЕСТВО: извлеки от 5 до 8 ключевых фактов в зависимости от насыщенности материала.
    
        ФОРМАТ ВЫВОДА:
        - Каждый факт начинается с дефиса "-" и новой строки
        - Факт формулируется как одно полное предложение (допустимо 2-3 предложения, если информация сложная)
        - НЕ добавляй нумерацию, заголовки, комментарии или метаинформацию
        - НЕ пиши вводных фраз типа "Вот факты:" или "Список фактов:"
    
        ПРИМЕР КОРРЕКТНОГО ВЫВОДА:
        - Производная функции f(x) обозначается как f'(x) и показывает скорость изменения функции в данной точке
        - Геометрический смысл производной — тангенс угла наклона касательной к графику функции в заданной точке
        - Производная константы равна нулю, так как константа не изменяется
        - Правило дифференцирования суммы: (f+g)' = f' + g'
        - Производная степенной функции x^n равна n·x^(n-1)
    
        Анализируй лекцию внимательно, выделяя фундаментальные знания, которые формируют понимание темы."""),
        ("user", "Текст лекции:\n\n{lecture_text}\n\nИзвлеки ключевые факты согласно заданным критериям:")
    ])
        chain = system_prompt | llm_parser
    
        response = chain.invoke({"lecture_text": lecture_text})
        facts_text = response.content
        facts = [
            line.strip().lstrip("- ").lstrip("• ")
            for line in facts_text.split("\n")
            if line.strip() and (line.strip().startswith("-") or line.strip().startswith("•"))
        ]
    
    
        state["key_facts"] = facts
        state["messages"].append(AIMessage(content=f"Parser Agent извлек {len(facts)} фактов"))
        state["current_step"] = "parsing_complete"
    
        return state
    
    print("\n" + "="*60)
    print("ТЕСТИРОВАНИЕ PARSER AGENT")
    print("="*60)
    
    result_state = parser_agent(test_state)
    print("\n Result:")
    for fact in result_state["key_facts"]:
        print(f"  ✓ {fact}")